#Final Project

Details:
*   Sign Language Dataset for ASL Numbers and Operators
*   CNN for recognizing hand sign
*   OpenCV to capture hand gestures
*   use trained model to recognize hand gestures
*   recognized signs to form mathematical expression
*   evaluate expression

In [2]:
!pip install opendatasets
!pip install pandas
!pip install opencv-python
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 35.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.3 which is incompatible.


In [3]:
import opendatasets as od
import pandas
import cv2
import os
import numpy as np
import mediapipe as mp

In [4]:
# od.download("https://www.kaggle.com/datasets/gti-upm/leapgestrecog/data")
od.download("https://www.kaggle.com/datasets/rayeed045/american-sign-language-digit-dataset/")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: divinelavente
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/rayeed045/american-sign-language-digit-dataset


100%|██████████| 288M/288M [00:04<00:00, 64.0MB/s]


In [ ]:
# import shutil
# directory_to_delete = '/content/'
# shutil.rmtree(directory_to_delete)
# print("Dataset deleted successfully.")

Dataset deleted successfully.


In [5]:
# Check the current working directory
print("Current working directory:", os.getcwd())

# List the contents of the current working directory
print("Contents of the current directory:", os.listdir())


Current working directory: /content
Contents of the current directory: ['.config', 'american-sign-language-digit-dataset', 'sample_data']


In [6]:
base_path = "/content/american-sign-language-digit-dataset"

In [17]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.7)

def extract_keypoints(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = hands.process(image_rgb)
    if result.multi_hand_landmarks:
        hand_landmarks = result.multi_hand_landmarks[0]
        keypoints = []
        for lm in hand_landmarks.landmark:
            keypoints.extend([lm.x, lm.y, lm.z])
        return keypoints
    return None

imagepaths = []
for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.endswith(('.png', '.jpg', '.jpeg')):  # Add any other image extensions if necessary
            imagepaths.append(os.path.join(root, file))

X = []  # Keypoints data
y = []  # Labels

for path in imagepaths:
    img = cv2.imread(path)
    if img is None:
        print(f"Error: The image at path '{path}' could not be read.")
        continue
    keypoints = extract_keypoints(img)
    if keypoints:
        X.append(keypoints)
        try:
            folder_name = path.split("/")[-2]
            label = ''.join(filter(str.isdigit, folder_name))
            y.append(int(label))
        except ValueError as ve:
            print(f"Error: Unable to convert label to integer in path '{path}': {ve}")
            continue

X = np.array(X)
y = np.array(y)

print("Keypoints extracted: ", len(X))
print("Labels extracted: ", len(y))

# Save keypoints and labels if needed
np.save('X_keypoints.npy', X)
np.save('y_labels.npy', y)


Keypoints extracted:  4303
Labels extracted:  4303


In [ ]:

# # Collecting all image paths
# imagepaths = []
# for root, dirs, files in os.walk(base_path):
#     for file in files:
#         if file.endswith(('.png', '.jpg', '.jpeg')):  # Add any other image extensions if necessary
#             imagepaths.append(os.path.join(root, file))

# X = []  # Image data
# y = []  # Labels

# # Loop through image paths to load images and labels into arrays
# for path in imagepaths:
#     img = cv2.imread(path)  # Reads image and returns np.array
#     if img is None:
#         print(f"Error: The image at path '{path}' could not be read.")
#         continue
#     try:
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Converts into the correct colorspace (GRAY)
#         img = cv2.resize(img, (320, 120))  # Reduce image size so training can be faster
#         X.append(img)

#         # Processing label in image path
#         category = path.split("/")[-2]
#         label = int(category.split("_")[0][1])  # Adjusted for proper directory depth
#         y.append(label)
#     except Exception as e:
#         print(f"Error processing image at path '{path}': {e}")
#         continue

# # Turn X and y into np.array to speed up train_test_split
# X = np.array(X, dtype="uint8")
# X = X.reshape(len(X), 120, 320, 1)  # Needed to reshape so CNN knows it's different images
# y = np.array(y)

# print("Images loaded: ", len(X))
# print("Labels loaded: ", len(y))


ValueError: cannot reshape array of size 2304000000 into shape (20000,120,320,1)

In [18]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# load keypoints and labels
X_keypoints = np.load('X_keypoints.npy')
y_labels = np.load('y_labels.npy')

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_keypoints, y_labels, test_size=0.2, random_state=42)

# Define a simple neural network model
model = Sequential([
    Dense(128, activation='relu', input_shape=(63,)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')  # 10 classes for digits 0-9
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

Epoch 1/20
108/108 [==============================] - 1s 5ms/step - loss: 2.1677 - accuracy: 0.2205 - val_loss: 1.8425 - val_accuracy: 0.4669
Epoch 2/20
108/108 [==============================] - 0s 3ms/step - loss: 1.6690 - accuracy: 0.4262 - val_loss: 1.2044 - val_accuracy: 0.7666
Epoch 3/20
108/108 [==============================] - 0s 3ms/step - loss: 1.1712 - accuracy: 0.6238 - val_loss: 0.7015 - val_accuracy: 0.9024
Epoch 4/20
108/108 [==============================] - 0s 3ms/step - loss: 0.8307 - accuracy: 0.7377 - val_loss: 0.4015 - val_accuracy: 0.9524
Epoch 5/20
108/108 [==============================] - 0s 2ms/step - loss: 0.5974 - accuracy: 0.8234 - val_loss: 0.2328 - val_accuracy: 0.9907
Epoch 6/20
108/108 [==============================] - 0s 3ms/step - loss: 0.4372 - accuracy: 0.8774 - val_loss: 0.1540 - val_accuracy: 0.9942
Epoch 7/20
108/108 [==============================] - 0s 3ms/step - loss: 0.3582 - accuracy: 0.9001 - val_loss: 0.0986 - val_accuracy: 0.9942
Epoch 

In [19]:
from tensorflow.keras.models import save_model
from google.colab import files

save_model(model, 'hand_gesture_model.h5')
files.download('hand_gesture_model.h5')

<ipython-input-19-451c0da75a47>:4: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(model, 'hand_gesture_model.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>